In [1]:
from keras import layers

from models import create_model
from models.layers import RandomSwitch
from utils import plot_metrics
from utils.data import load_data, over_sampling, random_resample

data_path = 'data/chest_xray'

2025-02-11 15:47:40.110159: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-11 15:47:40.110527: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-11 15:47:40.112418: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-11 15:47:40.117557: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739285260.126565  114226 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739285260.12

In [2]:
resize = (224, 224)

In [3]:
(x, y) = load_data(data_path, resize=resize, resample=random_resample)

In [4]:
NEG_IDX = 0
POS_IDX = 1
TOTAL   = len(y)
N       = len(list(filter(lambda j: j[NEG_IDX] > 0, y)))
P       = len(list(filter(lambda j: j[POS_IDX] > 0, y)))
print(f'#Positives = {P} ({P/TOTAL*100:.2f}%)')
print(f'#Negatives = {N} ({N/TOTAL*100:.2f}%)')

#Positives = 4273 (72.97%)
#Negatives = 1583 (27.03%)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.2, random_state=42)
total_train = len(Y_train)
N_train     = len(list(filter(lambda j: j[NEG_IDX] > 0, Y_train)))
P_train     = len(list(filter(lambda j: j[POS_IDX] > 0, Y_train)))
print(f'#Positives (train) = {P_train} ({P_train/total_train*100:.2f}%)')
print(f'#Negatives (train) = {N_train} ({N_train/total_train*100:.2f}%)')
total_val   = len(Y_val)
N_val       = len(list(filter(lambda j: j[NEG_IDX] > 0, Y_val)))
P_val       = len(list(filter(lambda j: j[POS_IDX] > 0, Y_val)))
print(f'#Positives (val)   = {P_val} ({P_val/total_val*100:.2f}%)')
print(f'#Negatives (val)   = {N_val} ({N_val/total_val*100:.2f}%)')

#Positives (train) = 3408 (72.76%)
#Negatives (train) = 1276 (27.24%)
#Positives (val)   = 865 (73.81%)
#Negatives (val)   = 307 (26.19%)


In [6]:
xs_extras, ys_extras = over_sampling(X_train, Y_train)
total   = len(ys_extras)
N_extras       = len(list(filter(lambda j: j[NEG_IDX] > 0, ys_extras)))
P_extras       = len(list(filter(lambda j: j[POS_IDX] > 0, ys_extras)))
print(f'#Positives (val)   = {P_extras} ({P_extras/total*100:.2f}%)')
print(f'#Negatives (val)   = {N_extras} ({N_extras/total*100:.2f}%)')

#Positives (val)   = 3408 (50.00%)
#Negatives (val)   = 3408 (50.00%)


In [7]:
tr = layers.RandomTranslation(0.2, 0.2, data_format='channels_last')
rr = layers.RandomRotation(0.2, data_format='channels_last')
rb = layers.RandomBrightness(0.2, data_format='channels_last')
rc = layers.RandomContrast(0.2, data_format='channels_last')
rz = layers.RandomZoom(0.2, 0.2, data_format='channels_last')
p1 = layers.Pipeline([rr, rz])
p2 = layers.Pipeline([tr, rr])
p3 = layers.Pipeline([rr, rc])
data_augmentation = RandomSwitch([tr, rr, rb, rc, rz, p1, p2, p3], 0.7)

2025-02-11 15:47:44.185273: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
v1 = create_model('resnet_v1', resize=resize, data_augmentation=data_augmentation)

/home/francesco/Documents/Università/ML/t5/.venv/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
from keras import losses, metrics, optimizers

METRICS = [
    metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
    # metrics.MeanSquaredError(name='Brier score'),
    # metrics.TruePositives(name='tp'),
    # metrics.FalsePositives(name='fp'),
    # metrics.TrueNegatives(name='tn'),
    # metrics.FalseNegatives(name='fn'),
    # metrics.BinaryAccuracy(name='accuracy'),
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall'),
    metrics.AUC(name='auc'),
    metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

v1.compile(optimizer=optimizers.Adam(), loss=losses.BinaryCrossentropy(), metrics=METRICS)
v1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_switch (RandomSwitch)    │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ stem_v1 (StemV1)                │ (None, 11, 11, 256)    │       651,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_a_vx (InceptionAVx)   │ (None, 11, 11, 256)    │        77,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_a_vx_1 (InceptionAVx) │ (None, 11, 11, 256)    │        77,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_a_vx_2 (InceptionAVx) │ (None, 11, 11, 256)    │        77,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_a_vx_3 (InceptionAVx) │ (None, 11, 11, 256)    │        77,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_a_vx_4 (InceptionAVx) │ (None, 11, 11, 256)    │        77,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reduction_a (ReductionA)        │ (None, 5, 5, 896)      │     1,709,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx (InceptionWVx)   │ (None, 5, 5, 896)      │       689,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_1 (InceptionWVx) │ (None, 5, 5, 896)      │       689,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_2 (InceptionWVx) │ (None, 5, 5, 896)      │       689,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_3 (InceptionWVx) │ (None, 5, 5, 896)      │       689,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_4 (InceptionWVx) │ (None, 5, 5, 896)      │       689,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_5 (InceptionWVx) │ (None, 5, 5, 896)      │       689,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_6 (InceptionWVx) │ (None, 5, 5, 896)      │       689,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_7 (InceptionWVx) │ (None, 5, 5, 896)      │       689,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_8 (InceptionWVx) │ (None, 5, 5, 896)      │       689,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_9 (InceptionWVx) │ (None, 5, 5, 896)      │       689,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reduction_b_vx (ReductionBVx)   │ (None, 2, 2, 1792)     │     3,344,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_10               │ (None, 2, 2, 1792)     │     1,600,000 │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_11               │ (None, 2, 2, 1792)     │     1,600,000 │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_12               │ (None, 2, 2, 1792)     │     1,600,000 │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 20,989,746 (80.07 MB)

 Trainable params: 20,989,746 (80.07 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
from keras import callbacks

EPOCHS = 100
BATCH_SIZE = 128
early_stopping = callbacks.EarlyStopping(
    verbose=1,
    patience=10,
    restore_best_weights=True)

In [13]:
history = v1.fit(
    xs_extras,
    ys_extras,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(X_val, Y_val),
    validation_freq=1)

Epoch 1/100


/home/francesco/Documents/Università/ML/t5/.venv/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'inception_a_vx', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/home/francesco/Documents/Università/ML/t5/.venv/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'inception_a_vx_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/home/francesco/Documents/Università/ML/t5/.venv/lib/python3.12/site-pac

 6/54 ━━━━━━━━━━━━━━━━━━━━ 35s 732ms/step - auc: 0.4853 - cross entropy: 0.7766 - loss: 0.7766 - prc: 0.4805 - precision: 0.4708 - recall: 0.4708

KeyboardInterrupt: 

In [ ]:
plot_metrics(history, metrics=['loss', 'precision', 'recall', 'auc'])